<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/S1mpleTransformerMachineTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Machine Tranalation

# reference
# https://simpletransformers.ai/docs/installation/
# https://towardsdatascience.com/how-to-train-an-mt5-model-for-translation-with-simple-transformers-30ba5fa66c5f
# https://huggingface.co/models

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

!pip install simpletransformers
!pip install torch
!pip install sacrebleu

# for marian model
!pip install sacremoses

In [ ]:
# config model

import logging
import torch
import pandas as pd

# -----enable one of this-----
# # for mt5 model
# from simpletransformers.t5 import T5Model, T5Args
# # for seq2seq model
# from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# load dataset
train_df = pd.read_csv('/content/gdrive/MyDrive/train.csv').astype(str)
eval_df = pd.read_csv('/content/gdrive/MyDrive/eval.csv').astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

# -----enable one of this-----
# # model args for mt5 model
# model_args = T5Args()
# # model args for seq2seq model
# model_args = Seq2SeqArgs()

# config model
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 8
# model_args.learning_rate = 1e-3 #optional
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "Machine Translation"

device = torch.cuda.is_available()

# -----enable one of this-----
# # mt5
# model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=device)
# # mbart
# model = Seq2SeqModel("mbart", "facebook/mbart-large-50", args=model_args, use_cuda=device)
# # marian
# model = Seq2SeqModel("marian", "Helsinki-NLP/opus-mt-th-en", args=model_args, use_cuda=device)

# website for more model
# https://huggingface.co/models?sort=downloads

In [ ]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

# Evaluate the model
result = model.eval_model(eval_df)

In [ ]:
# load model

import logging
import sacrebleu
import pandas as pd

# -----enable one of this-----
# # for mt5 model
# from simpletransformers.t5 import T5Model, T5Args
# # for seq2seq model
# from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# -----enable one of this-----
# # model args for mt5 model
# model_args = T5Args()
# # model args for seq2seq model
# model_args = Seq2SeqArgs()

model_args.max_length = 256
model_args.length_penalty = 1
model_args.num_beams = 10

# -----enable one of this-----
# # mt5
# model = T5Model("mt5", "/content/outputs/best_model", args=model_args)
# # mbart
# model = Seq2SeqModel("mbart", "/content/outputs/best_model", args=model_args)
# # marian
# model = Seq2SeqModel("marian", "/content/outputs/best_model", args=model_args)

In [ ]:
# calculate bleu score

test = pd.read_csv("/content/gdrive/MyDrive/test.csv").astype(str)

thai_truth = [test.loc[test["prefix"] == "translate english to thai"]["target_text"].tolist()]
to_thai = test.loc[test["prefix"] == "translate english to thai"]["input_text"].tolist()
english_truth = [test.loc[test["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = test.loc[test["prefix"] == "translate thai to english"]["input_text"].tolist()

english_preds = model.predict(to_english)
then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)
thai_preds = model.predict(to_thai)
enth_bleu = sacrebleu.corpus_bleu(thai_preds, thai_truth)
print("English to Thai: ", enth_bleu.score)

In [ ]:
# predict test

to_predict = ['โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ',
              'และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก',
              'โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน',
              'ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน',
              'และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน',
              'แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)',
              'ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน',
              'และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์',
              'ในขั้นตอนการเลือกแบบจําลองการเรียนรู้เชิงลึกเพื่อนําไปประมวลผลบนเครื่อง Jetson Nano']
predictions = model.predict(to_predict)
for i, info in enumerate(predictions):
  print(to_predict[i] + '\n' + info + '\n')

Generating outputs:   0%|          | 0/2 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/9 [00:00<?, ?it/s]

โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ
The mechanical work is to develop a detector system.

และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก
And classification of vegetables from the soil leaves images with detailed

โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน
The using the Json Nano machine to processed the images from the camera in the

ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน
According to the developed a detailed learning model, there was collected a

และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน
And create a variation sign of the gong.

แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)
And the dataset was used to apply to a simulation to detect object in the

ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน
It can be observed that there was a certain variation of kaffir lime

และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์
And when the system detects the fault, it sends 